In [1]:
from tensorflow import keras
keras.__version__

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'2.1.6-tf'

In [2]:
import tensorflow as tf
tf.__version__

'1.12.0'

# Paths, Datasets and Labels

In [4]:
from scipy.io import loadmat
from os.path import join

prefix = '../../Stanford_Dog_Breed'
imgfix = 'Images'
anofix = 'Annotation'

## Read the dataset

In [6]:
train = loadmat(join(prefix, 'train_list.mat'))
test = loadmat(join(prefix, 'test_list.mat'))

# L_annotation = L['annotation_list'];
# L_file = L['file_list'];
# L = L['labels'];

ftrain = train['file_list']
ftest = test['file_list']

ltrain = train['labels']
ltest = test['labels']

ntrain = ltrain.shape[0]
ntest = ltest.shape[0]

print('Size of the training set:\t', ntrain)
print('Size of the testing set: \t', ntest)

Size of the training set:	 12000
Size of the testing set: 	 8580


## Catagory 2 Index Table

In [7]:
DogBreed = {}
for i in range(1, ntrain):
    l = ltrain[i][0]
    c = ftrain[i][0][0].split('/')[0]
    c = c.split('-')[1]
    if c not in DogBreed:
        DogBreed[c] = l
        print(c, l)

Chihuahua 1
Japanese_spaniel 2
Maltese_dog 3
Pekinese 4
Shih 5
Blenheim_spaniel 6
papillon 7
toy_terrier 8
Rhodesian_ridgeback 9
Afghan_hound 10
basset 11
beagle 12
bloodhound 13
bluetick 14
black 15
Walker_hound 16
English_foxhound 17
redbone 18
borzoi 19
Irish_wolfhound 20
Italian_greyhound 21
whippet 22
Ibizan_hound 23
Norwegian_elkhound 24
otterhound 25
Saluki 26
Scottish_deerhound 27
Weimaraner 28
Staffordshire_bullterrier 29
American_Staffordshire_terrier 30
Bedlington_terrier 31
Border_terrier 32
Kerry_blue_terrier 33
Irish_terrier 34
Norfolk_terrier 35
Norwich_terrier 36
Yorkshire_terrier 37
wire 38
Lakeland_terrier 39
Sealyham_terrier 40
Airedale 41
cairn 42
Australian_terrier 43
Dandie_Dinmont 44
Boston_bull 45
miniature_schnauzer 46
giant_schnauzer 47
standard_schnauzer 48
Scotch_terrier 49
Tibetan_terrier 50
silky_terrier 51
soft 52
West_Highland_white_terrier 53
Lhasa 54
flat 55
curly 56
golden_retriever 57
Labrador_retriever 58
Chesapeake_Bay_retriever 59
German_short 60


## Helper Functions

In [8]:
def gen_path(idx):
    try:    path = join(prefix, imgfix, ftest[idx][0][0])
    except: path = join(prefix, imgfix, ftest[-1][0][0])      
    return  path

In [9]:
def gen_labl(idx):
    try:    labl = ltest[idx][0]
    except: labl = ltest[-1][0]
    return  labl

# Usage exmaples for image classification models

## Models 

    1. ResNet50
    2. DenseNet
    3. VGG19
    4. InceptionV3

## Calssify ImageNet classes with ResNet50

In [10]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

Using TensorFlow backend.


### dome

In [32]:
idx = np.random.choice(range(ntest))
path = gen_path(idx)
labl = gen_labl(idx)

print('Ground:', path.split('/')[4].split('-')[1])

img = image.load_img(path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
preds = decode_predictions(preds, top=3)[0];

# decode the results into a list of tuples (class, description, probability)
print('Predicted:', preds)

for pred in preds:
    c = pred[1];
    if c not in DogBreed: continue
    lb = DogBreed[c]
    if lb == labl: 
        print('Found the class with', c, 'and probability', pred[2])
    


Ground: French_bulldog
Predicted: [('n02108915', 'French_bulldog', 0.93131804), ('n02096585', 'Boston_bull', 0.06433643), ('n02108089', 'boxer', 0.0027737769)]
Found the class with French_bulldog and probability 0.93131804


In [34]:
X = []

for idx in range(ntest):
    path = gen_path(idx)
    labl = gen_labl(idx)
    try:
        img = image.load_img(path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        X.append(preprocess_input(x))
    except:
        print(idx)
    
XTest = np.vstack(X)
YTest = model.predict(XTest)

In [35]:
print(YTest.shape)

(8580, 1000)


In [39]:
true = 0
top = 1

for idx in range(ntest):
    pred = decode_predictions(YTest[idx:idx+1,:], top=top)[0]
    for k in range(top):
        if pred[k][1] not in DogBreed:
            continue
        labl = DogBreed[pred[k][1]]
        if labl == gen_labl(idx):
            true += 1
            break


print(true)
print(true / ntest)

6541
0.7623543123543124


In [40]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

## Extract features with VGG16

In [26]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

img_path = '../../Stanford_Dog_Breed/Images/n02085620-Chihuahua/n02085620_10074.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

In [28]:
print(features.shape)
print(decode_predictions(preds, top=3)[0])

(1, 7, 7, 512)
[('n02085620', 'Chihuahua', 0.99632937), ('n02087046', 'toy_terrier', 0.00053993583), ('n02112018', 'Pomeranian', 0.00050796004)]


## Extract features from an arbitrary intermediate layer with VGG19

In [29]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

img_path = '../../Stanford_Dog_Breed/Images/n02085620-Chihuahua/n02085620_10074.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)

574717952/574710816 [==============================] - 14s 0us/step


In [30]:
print(block4_pool_features.shape)

(1, 14, 14, 512)


## Fine-tune InceptionV3 on a new set of classes

In [31]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
